In [4]:
from pathlib import Path
from sys import stdout

import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import sympy
from scipy.constants import Boltzmann, electron_volt
from sympy import symbols
from sympy.vector import CoordSys3D
from pymatgen import units
from pymatgen.transformations.standard_transformations import SupercellTransformation
from ruamel.yaml import YAML
from sklearn.linear_model import LinearRegression

from neighbormodels.structure import from_file
from neighbormodels.neighbors import count_neighbors
from neighbormodels.interactions import build_model

pd.set_option("display.colheader_justify", "left")
pd.set_option("display.html.border", 0)
html_table_style = {"selector": "th", "props": [("text-align", "left")]}
yaml = YAML()

kB = 1000 * Boltzmann / electron_volt

In [6]:
fe_cif_filepath = "data/fe.cif"
fe_structure_2atom = from_file(structure_file=fe_cif_filepath)
rotate_cell_45_degrees = SupercellTransformation(
    scaling_matrix=[[1, 1, 0],
                    [1, -1, 0],
                    [0, 0, 1]],
)
fe_structure_4atom = rotate_cell_45_degrees.apply_transformation(fe_structure_2atom)

fe_magnetic_patterns = yaml.load(Path("data/fe_magnetic_patterns.yml"))

In [7]:
fe_neighbor_data = count_neighbors(cell_structure=fe_structure_4atom, r=3)
fe_exchange_model = build_model(magnetic_patterns=fe_magnetic_patterns["4atoms"], neighbor_data=fe_neighbor_data)

In [8]:
fe_exchange_model

,pattern,J1,J2
0,a-type,-8.0,6.0
1,f-type,8.0,6.0
2,g-type,0.0,-2.0
3,nm,0.0,0.0


In [19]:
fe_dft_energies = pd.read_csv(filepath_or_buffer="data/fe_qe_energies.csv") 
fe_dft_energies

,pattern,num_sites,total_energy
0,nm,4,-1315.548748
1,f-type,4,-1315.550599
2,a-type,4,-1315.579499
3,g-type,4,-1315.578555


In [21]:
fe_model_matrix = fe_dft_energies \
    .assign(energy = lambda x:
        (x["total_energy"] -
         np.float64(x.query("pattern == 'nm'").loc[:, "total_energy"])) *
        1000 * units.Ha_to_eV / x["num_sites"]) \
    .merge(fe_exchange_model, on=["pattern"]) \
    .query("pattern != 'nm'") \
    .assign(J1 = lambda x: x["J1"]) \
    .assign(J2 = lambda x: x["J2"]) \
    .loc[:, ["pattern", "energy", "J1", "J2"]]
fe_model_matrix

,pattern,energy,J1,J2
1,f-type,-12.594790,8.0,6.0
2,a-type,-209.193789,-8.0,6.0
3,g-type,-202.772446,0.0,-2.0


In [22]:
smf_exchange_fit = smf.ols(data=fe_model_matrix, formula="energy ~ J1 + J2").fit()
smf_exchange_parameters = pd.DataFrame(smf_exchange_fit.params, columns=["statsmodels"])

In [23]:
lm = LinearRegression()
lm_exchange_fit = lm.fit(X=fe_model_matrix[["J1", "J2"]], y=fe_model_matrix["energy"])
lm_exchange_parameters = pd.DataFrame({
    "sklearn": [lm_exchange_fit.intercept_, lm_exchange_fit.coef_[0], lm_exchange_fit.coef_[1]]},
    index=["Intercept", "J1", "J2"],
)

In [24]:
smf_exchange_parameters \
    .join(lm_exchange_parameters) \
    .reset_index() \
    .rename({"index": "parameter"}, axis=1) \
    .style.set_table_styles(html_table_style) \
    .hide_index()

parameter,statsmodels,sklearn
Intercept,-179.803,-179.803
J1,12.2874,12.2874
J2,11.4848,11.4848
